## Segmenting and Clustering Neighborhoods in Toronto

In [1]:
import pandas as pd
import numpy as np
import requests

In [2]:
# Grab the table on the wikipedia page.
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]

In [3]:
# When Borough is 'Not assigned', abandon them.
df = df[df.Borough!='Not assigned']

In [4]:
# When Neighbourhood is 'Not assigned', replace it with Borough.
for i in range(df.shape[0]):
    if df.iloc[i,2]=='Not assigned':
        df.iloc[i,2]=df.iloc[i,1]

### Add Latitude and Longitude for each Postcode.

In [5]:
# Read csv to df_geo.
df_geo = pd.read_csv('Geospatial_Coordinates.csv')
# Rename the column name of df_geo, so that it can merge with df.
df_geo.rename(columns={'Postal Code':'Postcode'},inplace=True)

In [6]:
# Merge df and df_geo.
neighborhoods = pd.merge(df,df_geo,how='left',on='Postcode')
neighborhoods = neighborhoods.drop(columns='Postcode')

In [7]:
neighborhoods = neighborhoods[neighborhoods['Borough'].str.contains('Toronto')]
neighborhoods.rename(columns={'Neighbourhood':'Neighborhood'},inplace=True)
neighborhoods.index=[i for i in range(neighborhoods.shape[0])]
toronto_data = neighborhoods
toronto_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Downtown Toronto,Harbourfront,43.654260,-79.360636
1,Downtown Toronto,Regent Park,43.654260,-79.360636
2,Downtown Toronto,Ryerson,43.657162,-79.378937
3,Downtown Toronto,Garden District,43.657162,-79.378937
4,Downtown Toronto,St. James Town,43.651494,-79.375418


In [8]:
# Get the geo of Toronto.
# from geopy.geocoders import Nominatim
# address = 'Toronto'
# geolocator = Nominatim(user_agent="ny_explorer")
# location = geolocator.geocode(address)
# latitude = location.latitude
# longitude = location.longitude
# print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))
# The geograpical coordinate of Toronto are 43.653963, -79.387207.
latitude = 43.653963
longitude = -79.387207

In [9]:
CLIENT_ID = 'IOV2GAGHT5QIZDIKYCJIBJTJUWP2QGO1EKICQVTC2NO1IALU' # your Foursquare ID
CLIENT_SECRET = 'ARNCSIMQGAQ0AIVPJTSTR0MWXBTV4MDHIHNOX0YTMUEYQUAP' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 #Venues count limit

In [10]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Harbourfront
Regent Park
Ryerson
Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide
King
Richmond
Dovercourt Village
Dufferin
Harbourfront East
Toronto Islands
Union Station
Little Portugal
Trinity
The Danforth West
Riverdale
Design Exchange
Toronto Dominion Centre
Brockton
Exhibition Place
Parkdale Village
The Beaches West
India Bazaar
Commerce Court
Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North
Forest Hill West
High Park
The Junction South
North Toronto West
The Annex
North Midtown
Yorkville
Parkdale
Roncesvalles
Davisville
Harbord
University of Toronto
Runnymede
Swansea
Moore Park
Summerhill East
Chinatown
Grange Park
Kensington Market
Deer Park
Forest Hill SE
Rathnelly
South Hill
Summerhill West
CN Tower
Bathurst Quay
Island airport
Harbourfront West
King and Spadina
Railway Lands
South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown
St. James Town
First Canadian Place
Underground city


In [11]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

In [12]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Café,Thai Restaurant,Bar,Steakhouse,Bakery,Burger Joint,Hotel,Sushi Restaurant,American Restaurant
1,Bathurst Quay,Airport Lounge,Airport Service,Airport Terminal,Boutique,Sculpture Garden,Harbor / Marina,Airport,Airport Food Court,Airport Gate,Plane
2,Berczy Park,Coffee Shop,Cocktail Bar,Café,Farmers Market,Seafood Restaurant,Cheese Shop,Steakhouse,Bakery,Italian Restaurant,Restaurant
3,Brockton,Coffee Shop,Café,Breakfast Spot,Gym / Fitness Center,Furniture / Home Store,Convenience Store,Climbing Gym,Restaurant,Caribbean Restaurant,Burrito Place
4,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Gym / Fitness Center,Garden,Pizza Place,Recording Studio,Restaurant,Butcher,Burrito Place,Skate Park,Brewery


In [13]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 2, 0, 0, 0, 2, 0, 0, 0, 0], dtype=int32)

In [14]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,Harbourfront,43.654260,-79.360636,0,Coffee Shop,Café,Bakery,Pub,Park,Mexican Restaurant,Breakfast Spot,Theater,Italian Restaurant,Chocolate Shop
1,Downtown Toronto,Regent Park,43.654260,-79.360636,0,Coffee Shop,Café,Bakery,Pub,Park,Mexican Restaurant,Breakfast Spot,Theater,Italian Restaurant,Chocolate Shop
2,Downtown Toronto,Ryerson,43.657162,-79.378937,0,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Middle Eastern Restaurant,Restaurant,Thai Restaurant,Pizza Place,Italian Restaurant,Japanese Restaurant
3,Downtown Toronto,Garden District,43.657162,-79.378937,0,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Middle Eastern Restaurant,Restaurant,Thai Restaurant,Pizza Place,Italian Restaurant,Japanese Restaurant
4,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Restaurant,Café,Bakery,Breakfast Spot,Hotel,Pizza Place,Gastropub,Italian Restaurant,Park


In [15]:
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [16]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Harbourfront,Coffee Shop,Café,Bakery,Pub,Park,Mexican Restaurant,Breakfast Spot,Theater,Italian Restaurant,Chocolate Shop
1,Regent Park,Coffee Shop,Café,Bakery,Pub,Park,Mexican Restaurant,Breakfast Spot,Theater,Italian Restaurant,Chocolate Shop
2,Ryerson,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Middle Eastern Restaurant,Restaurant,Thai Restaurant,Pizza Place,Italian Restaurant,Japanese Restaurant
3,Garden District,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Middle Eastern Restaurant,Restaurant,Thai Restaurant,Pizza Place,Italian Restaurant,Japanese Restaurant
4,St. James Town,Coffee Shop,Restaurant,Café,Bakery,Breakfast Spot,Hotel,Pizza Place,Gastropub,Italian Restaurant,Park
5,The Beaches,Coffee Shop,Health Food Store,Pub,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
6,Berczy Park,Coffee Shop,Cocktail Bar,Café,Farmers Market,Seafood Restaurant,Cheese Shop,Steakhouse,Bakery,Italian Restaurant,Restaurant
7,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Burger Joint,Bar,Bubble Tea Shop,Salad Place,Sushi Restaurant,Chinese Restaurant,Thai Restaurant
8,Christie,Grocery Store,Café,Park,Athletics & Sports,Nightclub,Italian Restaurant,Diner,Restaurant,Baby Store,Coffee Shop
9,Adelaide,Coffee Shop,Café,Thai Restaurant,Bar,Steakhouse,Bakery,Burger Joint,Hotel,Sushi Restaurant,American Restaurant


In [17]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
34,Forest Hill North,Trail,Jewelry Store,Park,Sushi Restaurant,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Women's Store
35,Forest Hill West,Trail,Jewelry Store,Park,Sushi Restaurant,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Women's Store
66,Rosedale,Park,Playground,Trail,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


In [18]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
59,CN Tower,Airport Lounge,Airport Service,Airport Terminal,Boutique,Sculpture Garden,Harbor / Marina,Airport,Airport Food Court,Airport Gate,Plane
60,Bathurst Quay,Airport Lounge,Airport Service,Airport Terminal,Boutique,Sculpture Garden,Harbor / Marina,Airport,Airport Food Court,Airport Gate,Plane
61,Island airport,Airport Lounge,Airport Service,Airport Terminal,Boutique,Sculpture Garden,Harbor / Marina,Airport,Airport Food Court,Airport Gate,Plane
62,Harbourfront West,Airport Lounge,Airport Service,Airport Terminal,Boutique,Sculpture Garden,Harbor / Marina,Airport,Airport Food Court,Airport Gate,Plane
63,King and Spadina,Airport Lounge,Airport Service,Airport Terminal,Boutique,Sculpture Garden,Harbor / Marina,Airport,Airport Food Court,Airport Gate,Plane
64,Railway Lands,Airport Lounge,Airport Service,Airport Terminal,Boutique,Sculpture Garden,Harbor / Marina,Airport,Airport Food Court,Airport Gate,Plane
65,South Niagara,Airport Lounge,Airport Service,Airport Terminal,Boutique,Sculpture Garden,Harbor / Marina,Airport,Airport Food Court,Airport Gate,Plane


In [19]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
49,Moore Park,Playground,Gym,Tennis Court,Restaurant,Doner Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop
50,Summerhill East,Playground,Gym,Tennis Court,Restaurant,Doner Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop


In [20]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,Roselawn,Garden,Pool,Women's Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
